First function in MFE Practical Work 2 assignment

Designed to take a daily price pandas series

Returns a pandas dataframe of mean, std_dev, skew, kurt: 

Sampled from daily, weekly, monthly and annual data

Points

 - Mean has small differences due to sampling mechanics

 - standard deviation differs slightly - most likely due to macro movements

 - need to check equations and scaling for skew and kurtosis

 - additionally check that we are resampling the returns data correctly

In [4]:
import numpy as np
import pandas as pd

def summary_statistics(prices):

    d_returns = prices.pct_change().dropna()
    w_returns = prices.resample("W-FRI").last().pct_change().dropna()
    m_returns = prices.resample("M").last().pct_change().dropna()
    q_returns = prices.resample("Q").last().pct_change().dropna()
    # Convert prices to returns

    # Enclosed function to find mean, std, skew and kurt: gives annualised stats
    def process(returns, n):
        returns_mu = returns.mean()                         # mean returns per time period
        annual_mu =  returns_mu * n                       # annualised mean returns

        returns_err = returns - returns_mu       
        returns_var = (returns_err ** 2).mean()             # average squared return ( 2nd moment)
        annual_var = returns_var * n                        # annual rescaled variance
        annual_std = np.sqrt(annual_var)                    # annual rescaled std deviation

        returns_mom3 = (returns_err ** 3).mean()
        returns_mom4 = (returns_err ** 4).mean()            # Third and Fourth moments


        annual_skew = (returns_mom3 / returns_var ** (3/2)) #/ np.sqrt(n)  # rescaled Skew and Kurtosis
        annual_kurt = (returns_mom4 / returns_var ** (4/2)) #/ n

        returns_stats = pd.Series([annual_mu, annual_std, annual_skew, annual_kurt], index = ["mean", "std", "skew", "kurt"])
        
        return returns_stats
         
    # Feed in daily, weekly, monthly and quarterly data
    d_stats = process(d_returns, n = 252).rename("Daily")
    w_stats = process(w_returns, n = 52 ).rename("Weekly")
    m_stats = process(m_returns, n = 12 ).rename("Monthly")
    q_stats = process(q_returns, n = 4  ).rename("Quarterly")

    # Concatenate the series into a dataframe
    stats = pd.concat([d_stats, w_stats, m_stats, q_stats], axis=1)

    return stats

In [5]:
aapl = pd.read_csv("data/aapl.csv", parse_dates=True, index_col="Date")
aapl_close = aapl["Close"]

sbry_1 = pd.read_csv("data/SBRY_1Y.csv", parse_dates=True, index_col="Date")
sbry_1 = sbry_1["Close"]

sbry_10 = pd.read_csv("data/SBRY_10Y.csv", parse_dates=True, index_col="Date")
sbry_10 = sbry_10["Close"]

FTSE = pd.read_csv("data/^FTSE.csv", parse_dates=True, index_col="Date")
FTSE = FTSE["Close"]

In [6]:
summary_statistics(aapl_close)

,Daily,Weekly,Monthly,Quarterly
mean,0.285387,0.305630,0.319264,0.316689
std,0.243743,0.243948,0.266882,0.296066
skew,-0.299724,-0.059524,-0.532603,-0.893037
kurt,7.643469,5.068484,3.265992,3.691122


In [7]:
summary_statistics(sbry_1)

,Daily,Weekly,Monthly,Quarterly
mean,-0.059793,-0.057119,-0.164591,0.025990
std,0.308372,0.298536,0.301015,0.251771
skew,-3.424303,-1.186042,-0.423066,-0.606182
kurt,34.493901,8.679968,2.692329,1.500000


In [8]:
summary_statistics(sbry_10)

,Daily,Weekly,Monthly,Quarterly
mean,-0.001567,-0.003904,-0.008727,-0.005305
std,0.249527,0.239150,0.216670,0.209488
skew,-0.130554,-0.248017,0.332873,0.579807
kurt,17.233344,5.667233,5.999706,4.500629


In [9]:
summary_statistics(FTSE)

,Daily,Weekly,Monthly,Quarterly
mean,0.126650,0.109794,0.097889,0.055890
std,0.117343,0.106281,0.097565,0.023308
skew,-0.403668,-0.187597,-0.984578,-0.644009
kurt,5.072695,2.709030,2.458230,1.500000


In [10]:
type(aapl_close)

pandas.core.series.Series

In [11]:
np.random.seed(3)
rng = pd.bdate_range('2015-01-01', periods=10000, freq='B')
returns = pd.Series(np.random.normal(0, 0.01, len(rng)), index=rng)
prices = (1+returns).cumprod()
summary_statistics(prices)


,Daily,Weekly,Monthly,Quarterly
mean,-0.070157,-0.073508,-0.071572,-0.068949
std,0.158074,0.153607,0.158098,0.159322
skew,0.028539,0.022837,0.120475,0.274052
kurt,3.003214,3.237223,3.052913,3.198883


In [12]:
summary_statistics(prices)

,Daily,Weekly,Monthly,Quarterly
mean,-0.070157,-0.073508,-0.071572,-0.068949
std,0.158074,0.153607,0.158098,0.159322
skew,0.028539,0.022837,0.120475,0.274052
kurt,3.003214,3.237223,3.052913,3.198883


In [13]:
from scipy.special import gammaln
from scipy.optimize import minimize

def students_dof(cheese):
    def std_t_loglik(nu, x):
        # These are fixed for now
        mu = 0
        sigma2 = 1
        sigma = np.sqrt(sigma2)


        a = gammaln((nu + 1) / 2)
        b = gammaln(nu / 2)
        c = np.sqrt(np.pi * (nu-2))
        d = ((nu + 1) / 2)
        e = (x - mu) **2
        f = sigma2 * (nu - 2)

        loglik = a - b - np.log(c) - np.log(sigma) - d * np.log(1 + e / f)
        return -(loglik.sum())
    def calc_kurt(series):
        mu = series.mean()
        err = series - mu
        var = (err**2).mean()
        mom4 = (err**4).mean()
        kurt = mom4 / var ** (4/2)
        return kurt
    starting_val = np.array([3]) 
    opt = minimize(std_t_loglik, starting_val, args=(cheese), bounds=[(2.01, 100)], options={"disp": True})
    mle = opt.x[0]
    if calc_kurt(cheese) > 4:
        moment = 3*(((calc_kurt(cheese) - 2) / (calc_kurt(cheese) - 4)))
    else:
        moment = float("inf")
    return mle, moment

In [14]:
np.random.seed(6)
df=3
returns2 = pd.Series(np.random.standard_t(df=df, size=15000)) / np.sqrt(df / (df - 2))

students_dof(returns2)


(3.036600555045798, 3.4821292418001035)